## <div align = center> Immersion à Data : *Rapport de synthèse*

In [2]:
# importations 

import pandas as pd
import numpy as np
import sklearn as sk

In [3]:
df = pd.read_csv("datas.csv", sep = ";")
df.describe()

,id,age,priors_count,is_recid,is_violent_recid,decile_score
count,11757.000000,11757.000000,11757.000000,11757.000000,11757.000000,11757.000000
mean,5879.000000,35.143319,3.082164,0.253806,0.075019,4.371268
std,3394.097892,12.022894,4.687410,0.558324,0.263433,2.877598
min,1.000000,18.000000,0.000000,-1.000000,0.000000,-1.000000
25%,2940.000000,25.000000,0.000000,0.000000,0.000000,2.000000
50%,5879.000000,32.000000,1.000000,0.000000,0.000000,4.000000
75%,8818.000000,43.000000,4.000000,1.000000,0.000000,7.000000
max,11757.000000,96.000000,43.000000,1.000000,1.000000,10.000000


In [4]:
# on transforme les dates en flotant

df['year_in'] = pd.DatetimeIndex(df['c_jail_in']).year
df['month_in'] = pd.DatetimeIndex(df['c_jail_in']).month
df['month_out'] = pd.DatetimeIndex(df['c_jail_out']).month
df['year_out'] = pd.DatetimeIndex(df['c_jail_out']).year

df['day_out'] = pd.DatetimeIndex(df['c_jail_out']).day
df['day_in'] = pd.DatetimeIndex(df['c_jail_in']).day

df['date_in'] = df['year_in'] + df['month_in']/12 + df['day_in']/365
df['date_out'] = df['year_out'] + df['month_out']/12 + df['day_out']/365
df['duree_prison'] = df['date_out'] - df['date_in']

In [5]:
# On transforme les catégories en flottant 

# Le sexe
df["is_Male"] = df["sex"] == "Male"
#Les catégories sociales
for column in df["social_cat"].unique():
    df[column] = (df["social_cat"] == column).astype(int)

# On supprime les colonnes associés au catégories texte
df.drop(["social_cat", "sex"], axis = 1, inplace = True)


In [7]:
# On supprime les valeurs valant - 1 dans les récidivistes 
# parce qu'on ne veut pas être biasié par les personnes qui n'ont jamais fait de prison

mask = df["is_recid"] != - 1
df = df[mask]

In [8]:
# on supprime toutes les personnes qui sont encore en prison à ce jour
# parce que l'on peut pas les labéliser par la récidive

mask = (df["date_out"] <= 2016.29)
df = df[mask]
df['duree_dehors'] = df['date_out'].max() - df['date_out']

In [9]:
new_df = df[['id', 'age', 'priors_count', 'is_recid', 'duree_dehors', 'duree_prison', "is_Male", 'Other', 'Farmer', 'Worker', 'OfficeWorker', 'Manager', 'CivilServant']]
new_df

,id,age,priors_count,is_recid,duree_dehors,duree_prison,is_Male,Other,Farmer,Worker,OfficeWorker,Manager,CivilServant
0,1,69,0,0,2.577854,0.002740,True,1,0,0,0,0,0
2,3,34,0,1,3.102511,0.025799,True,0,1,0,0,0,0
3,4,24,4,1,2.911187,0.002740,True,0,1,0,0,0,0
6,7,44,0,0,2.280137,0.003881,True,1,0,0,0,0,0
7,8,41,14,1,2.050457,0.016438,True,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11751,11752,63,5,1,3.044977,0.002740,True,0,1,0,0,0,0
11752,11753,47,1,0,2.472603,0.002740,True,1,0,0,0,0,0
11753,11754,22,3,1,2.816895,0.005479,True,0,0,1,0,0,0
11755,11756,29,2,0,2.805936,0.002740,True,0,0,1,0,0,0
